<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Replication-of-the-findings-in-the-paper-&quot;A-quantum-algorithm-for-gravitational-wave-matched-filtering&quot;,-by-S.-Gao-et-al." data-toc-modified-id="Replication-of-the-findings-in-the-paper-&quot;A-quantum-algorithm-for-gravitational-wave-matched-filtering&quot;,-by-S.-Gao-et-al.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Replication of the findings in the paper "A quantum algorithm for gravitational wave matched filtering", by S. Gao et al.</a></span><ul class="toc-item"><li><span><a href="#Proof-of-principle-for-template-matching-on-quantum-computer" data-toc-modified-id="Proof-of-principle-for-template-matching-on-quantum-computer-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Proof of principle for template matching on quantum computer</a></span><ul class="toc-item"><li><span><a href="#Grover's-Algorithm" data-toc-modified-id="Grover's-Algorithm-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Grover's Algorithm</a></span></li><li><span><a href="#Quantum-Counting" data-toc-modified-id="Quantum-Counting-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Quantum Counting</a></span></li></ul></li><li><span><a href="#Testing-Circuits" data-toc-modified-id="Testing-Circuits-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Testing Circuits</a></span></li></ul></li><li><span><a href="#CONSTRUCTING-NEW-GROVER-CIRCUIT" data-toc-modified-id="CONSTRUCTING-NEW-GROVER-CIRCUIT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>CONSTRUCTING NEW GROVER CIRCUIT</a></span></li></ul></div>

# Replication of the findings in the paper "A quantum algorithm for gravitational wave matched filtering", by S. Gao et al.

In [1]:
from qiskit import QuantumCircuit, assemble, Aer, QuantumRegister, ClassicalRegister, AncillaRegister, transpile
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
import math
import plotly.express as px
import numpy as np

## Proof of principle for template matching on quantum computer

This next section replicates the algorithm described in the paper "A quantum algorithm for gravitational wave matched filtering" in the section IV. At the moment, only the Grover's algorithm part is included, but the number of iterations necessary should be given through quantum counting, which we will incorporate soon.

### Grover's Algorithm

In [ ]:
# Making the circuit in the paper

n = 6 #number of qbits
q = 1 #number of qbits not used in the Grover's algo to allow for more template matches (every qbit taken out allow 2^q templates to match)

qrd = QuantumRegister(n, 'd') #Creates the data registry
qrt = QuantumRegister(n, 't') #Creates the template registry
anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

qc = QuantumCircuit(qrd, qrt, anc, cr) #Makes the circuit with these qbits as input

####### State initialization #########

z = [1, 0] # base vect |0>
o = [0, 1] # base vect |1>

init_st_data = [z,o,o,z,z,z] #data bit stream 011000 (this is what we will compare the templates to and hope it matches)

for i in range(n):
    qc.initialize(init_st_data[i], i) #initializes the the data registry into the above mentionned stream
    qc.h(n+i) #initializes the template into an equal amplitude superposition (with the Hadamard gates)
    
#initiation of ancillary to |->
qc.x(anc)
qc.h(anc)
    
#qc.draw() #Draws the initial circuit

In [ ]:
#we now create a Grover's Gate (oracle + diffuror) function that will output a circuit that applies 
#one cycle of Grover's iteration to the input qbits
def G_circ(n,q):
    
    #same as above, we recreate the input circuit, but without the classical bit registry, as this doesn't matter for a gate
    qrd = QuantumRegister(n, 'd')
    qrt = QuantumRegister(n, 't')
    anc = QuantumRegister(1, 'ancilla')
    gqc = QuantumCircuit(qrd, qrt, anc)
    
    ######## Oracle ##########
    # Still need to study a little the oracle to understand it totally, specifically the ancillary qbit action.
    
    # The first part is to apply CNOT gates to from the q to n data qbits onto the q to n template qbits.
    # 
    for i in range(q,n):
        gqc.cx(i,n+i)
    
    #This step creates the multi-control-NOT gate to create phase kickback
    gqc.x(range(n+q,2*n))
    gqc.mcx(list(range(n+q,2*n)), 2*n, mode='noancilla')
    gqc.x(range(n+q,2*n))   
    
    # I guess this step untagles the states of the qbits from data and template registry? 
    for i in range(q,n):
        gqc.cx(i,n+i)

    ######### Diffuser ###########
    #The diffuser has a standard shape that can just be used out of the box
    gqc.h(range(n,2*n))
    gqc.x(range(n,2*n))

    #It is difficult to do an actual multi-control-Z gate, so instead we make a MCX surrounded by H gates to transform it into a MCZ
    gqc.h(2*n-1)
    gqc.mct(list(range(n,2*n-1)), 2*n-1) 
    gqc.h(2*n-1)
    
#     Same thing than above with a different function
#     gqc.h(2*n-1)
#     gqc.mcx(list(range(n,2*n-1)), 2*n-1, mode='noancilla')
#     gqc.h(2*n-1)

    #The other symmetric side of the diffusor
    gqc.x(range(n,2*n))
    gqc.h(range(n,2*n))

    return gqc

In [ ]:
grit = G_circ(n,q).to_gate() #Transforms the circuit into a usable gate
grit.label = "Grover" #Names the gate

print(G_circ(n,q).draw(fold=-1)) #Schematic of the gate circuit

In [ ]:
#This is the repetition section, where we repeat the Grover gate a few times to improve the 
#probability of getting the expect answwers. The M is the number of iterations and at the moment it was 
#chosen thanks to the paper, but we will implement quantum counting as well bellow to showcase the method
M = 1
for i in range(M):
    qc.append(grit,range(2*n+1))

qc.measure(qrt,cr) #add the measurment gates to the template registry (theu go onto the classical bits)
qc.draw(fold=-1)

In [ ]:
#Here we create the simulation of the circuit

aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_sim)
shots = 2048 #we repeat the simulation 2048 times
qobj = assemble(transpiled_qc, shots=shots)
job = aer_sim.run(qobj)
hist = job.result().get_counts() #simulation output
#print(hist)

#We plot the results using plotly as a plotting tool
results = {'val':hist.keys(),'count':hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

In [ ]:
#Here we create the simulation of the circuit

aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_sim)
shots = 2*2048 #we repeat the simulation 2048 times
qobj = assemble(transpiled_qc, shots=shots)
job = aer_sim.run(qobj)
hist1 = job.result().get_counts() #simulation output
#print(hist)

#We plot the results using plotly as a plotting tool
results1 = {'val':hist.keys(),'count':hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results1, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

These results are what was expected, and are similar to the papers findings.
Seems that the ancillary being set to $|->$ does help the detection. (makes counts go from approx. 800 counts when initialized to $|0>$ to 1.7k counts with $|->$)

### Quantum Counting

In [2]:
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [ ]:
t = 4   # no. of counting qubits

cgrit = grit.control()
#with .control(nb_ctrl_qbits), the first values of the list are the controls
qft_dagger = qft(t).to_gate().inverse()
qft_dagger.label = "QFT†"

qrc = QuantumRegister(t, 'count')
qrd = QuantumRegister(n, 'd')
qrt = QuantumRegister(n, 't')
anc = QuantumRegister(1, 'ancilla')
cr = ClassicalRegister(t, 'c')

qc = QuantumCircuit(qrc, qrd, qrt, anc, cr)


# Initialize all qubits to |+>
for qubit in range(t+2*n):
    qc.h(qubit)
    
# #initiation of ancillary to |->
qc.x(anc)
qc.h(anc)


# Begin controlled Grover iterations
iterations = 1
for qubit in range(t):
    for i in range(iterations):
        qc.append(cgrit, [qubit] + [*range(t, t+2*n+1)])
    iterations *= 2
    
# Do inverse QFT on counting qubits
qc.append(qft_dagger, range(t))

# Measure counting qubits
qc.measure(range(t), range(t))

# Display the circuit
qc.draw(fold=-1)

In [ ]:
# Execute and see results
aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_sim)
qobj = assemble(transpiled_qc)
job = aer_sim.run(qobj)
hist = job.result().get_counts()

results = {'val':hist.keys(),'count':hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

We now need to divide our result by $2^n$ to get θ:

In [ ]:
measured_str = max(hist, key=hist.get)
measured_int = int(measured_str,2)
print("Register Output = %i" % measured_int)

theta = (measured_int/(2**t))*math.pi*2
print("Theta = %.5f" % theta)

N = 2**n
M = N * (math.sin(theta/2)**2)
print("No. of Solutions = %.1f" % (N-M))

k = math.pi/4*math.sqrt(N/(N-M))-0.5
print(f"The optimal number of iterations is: {k}")

## Testing Circuits

In [ ]:
from qiskit import QuantumCircuit, Aer, assemble
from math import pi
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex

In [ ]:
qc = QuantumCircuit(2)
qc.x(0)
qc.x(1)

qc.h(1)

# qc.cx(0,1)

# n=2
# qc.h(range(n,2*n))
# qc.x(range(n,2*n))

# #It is difficult to do an actual multi-control-Z gate, so instead we make a MCX surrounded by H gates to transform it into a MCZ
# qc.h(2*n-1)
# qc.mct(list(range(n,2*n-1)), 2*n-1) 
# qc.h(2*n-1)

# #     Same thing than above with a different function
# #     gqc.h(2*n-1)
# #     gqc.mcx(list(range(n,2*n-1)), 2*n-1, mode='noancilla')
# #     gqc.h(2*n-1)

# #The other symmetric side of the diffusor
# qc.x(range(n,2*n))
# qc.h(range(n,2*n))


display(qc.draw())
# See the result

svsim = Aer.get_backend('aer_simulator')
qc1 = qc.copy()
qc1.save_statevector()
final_state = svsim.run(qc1).result().get_statevector()
display(array_to_latex(final_state, prefix="\\text{Statevector} = "))
print(final_state.real)

In [ ]:
del(qc)
qc = QuantumCircuit(1)
qc.h(0)
# qc.x(0)

# qc.h(3)


display(qc.draw())
# See the result

svsim = Aer.get_backend('aer_simulator')
qc1 = qc.copy()
qc1.save_statevector()
final_state = svsim.run(qc1).result().get_statevector()
display(array_to_latex(final_state, prefix="\\text{Statevector} = "))
print(final_state.real)

# CONSTRUCTING NEW GROVER CIRCUIT

In [5]:
t = 4   # no. of counting qubits
n = 2

qcx = QuantumCircuit(4)
a = qcx.to_gate()
a.label = "U"

cgrit = a.control()
#with .control(nb_ctrl_qbits), the first values of the list are the controls
qft_dagger = qft(t).to_gate().inverse()
qft_dagger.label = "QFT†"

qrc = QuantumRegister(t, 'count')
qrd = QuantumRegister(n, 'd')
qrt = QuantumRegister(n, 't')

qc = QuantumCircuit(qrc, qrd, qrt)


# Initialize all qubits to |+>
for qubit in range(t+2*n):
    qc.h(qubit)


# Begin controlled Grover iterations
iterations = 1
for qubit in range(t):
    qc.append(cgrit, [qubit] + [*range(t, t+2*n)])
    
# Do inverse QFT on counting qubits
qc.append(qft_dagger, range(t))

# Display the circuit
print(qc.draw(output='latex_source',fold=-1))

% \documentclass[preview]{standalone}
% If the image is too large to fit on this documentclass use
\documentclass[draft]{beamer}
% img_width = 8, img_depth = 8
\usepackage[size=custom,height=12,width=19,scale=0.7]{beamerposter}
% instead and customize the height and width (in cm) to fit.
% Large images may run out of memory quickly.
% To fix this use the LuaLaTeX compiler, which dynamically
% allocates memory.
\usepackage[braket, qm]{qcircuit}
\usepackage{amsmath}
\pdfmapfile{+sansmathaccent.map}
% \usepackage[landscape]{geometry}
% Comment out the above line if using the beamer documentclass.
\begin{document}

\begin{equation*}
    \Qcircuit @C=1.0em @R=0.2em @!R {
	 	\lstick{ {count}_{0} :  } & \gate{\mathrm{H}} & \ctrl{4} & \qw & \qw & \qw & \multigate{3}{\mathrm{QFT†}} & \qw & \qw\\
	 	\lstick{ {count}_{1} :  } & \gate{\mathrm{H}} & \qw & \ctrl{3} & \qw & \qw & \ghost{\mathrm{QFT†}} & \qw & \qw\\
	 	\lstick{ {count}_{2} :  } & \gate{\mathrm{H}} & \qw & \qw & \ctrl{2} & \qw & \ghost